In [2]:
import datetime
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, \
                            mean_squared_error, median_absolute_error, accuracy_score
import numpy as np
#from pmdarima.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
#from pmdarima import auto_arima
import numpy as np
import sklearn.metrics
import sklearn

## Regressão linear
from sklearn.linear_model import LinearRegression

## Gradient Boosting
from xgboost import XGBRegressor

ModuleNotFoundError: No module named 'xgboost'

In [3]:
df_model = pd.read_csv('df_model_final.csv')
df_model = df_model.loc[df_model['NObs'] >= 16]

In [4]:
scoring = ['neg_mean_absolute_error', 'neg_mean_absolute_percentage_error']
refit_value = 'neg_mean_absolute_error'

## Regressão linear
lrParams = {"fit_intercept": [True, False], 
              "copy_X":[True, False], 
              "n_jobs":[1,2,None], 
              "positive":[True, False]}


## Gradient Boosting
GBSearchSpace = {"n_estimators": [20, 50], 
              "max_depth":[3,6], 
              "gamma":[0.01,0.1], 
              "learning_rate":[0.001,0.01,1]}

xgb = XGBRegressor(random_state=100)

In [5]:
def modelos(dataframe, var_expl):
    """
    Retorna a quantidade de itens que serão vendidos dado um preço
    
    params: dataframe -> Dataframe sobre o qual será feita a regressão linear
            var_expl -> recebe as variáveis explicativas (str ou listas de strings)
    """
    
    results = {'Ajuste':[], 'Previsoes':[]}
    
    results_reg = {'Serie':[], 'Intercept':[], 'Coef':[], 'MAE':[], 'MAPE':[]}
    #results_regGB = {'Serie':[], 'Intercept':[], 'Coef':[], 'MAE':[], 'MAPE':[]}
    
    df = pd.DataFrame(data=None, columns=df_model.columns)
    df['VendasPrevistas'] = ''
    df['NomeModelo'] = ''

    for comb in dataframe['UF_GrpProd'].unique():

        if isinstance(var_expl, str):
            X = np.array(dataframe.loc[dataframe["UF_GrpProd"] == comb][var_expl]).reshape(-1,1)
        else:
            X = np.array(dataframe.loc[dataframe["UF_GrpProd"] == comb][var_expl])

            Y = np.array(dataframe.loc[dataframe["UF_GrpProd"] == comb]["Vendas Reais Qtde"]).reshape(-1, 1)
            

            ## Linear Regression
            LRGridSearch = GridSearchCV(estimator = LinearRegression(),
                                       param_grid = lrParams,
                                       scoring = scoring,
                                       refit = refit_value)
            
            ## Gradient Boosting
            GS_xgb =  GridSearchCV(estimator = xgb,
                      param_grid = GBSearchSpace,
                      scoring = scoring, #definido na primeira versão
                      refit = refit_value,#definido na primeira versão
                      cv=4,
                      verbose = 0) #número de informações a serem impressas a cada iteração 
            
            
            ## Ajustando o modelo Linear Regression
            lr = LRGridSearch.fit(X, Y)
            
            ## Ajustando o modelo Gradient Boosting
            gb = GS_xgb.fit(X ,Y)
            
            
            
            ## Encontrando MAE de Linear Regression
            maeLR = lr.best_score_
            
            ## Encontrando MAE de Linear Regression
            maeGB = gb.best_score_
            
            
            dfteste = dataframe.loc[dataframe["UF_GrpProd"] == comb].copy()
            
            
            ## Decidindo qual modelo será utilizado para fazer a predição
            if maeLR > maeGB:
                predict = lr.predict(X)
                dfteste['NomeModelo'] = 'LR'
            
            else:
                predict = gb.predict(X)
                dfteste['NomeModelo'] = 'XGB'
            
            
            dfteste['VendasPrevistas'] = predict
            df = pd.concat([df, dfteste])
            
            mape = mean_absolute_percentage_error(dfteste["Vendas Reais Qtde"], dfteste["VendasPrevistas"])
            
            results_reg["Serie"].append(comb)
            results_reg["Intercept"].append(lr.best_estimator_.intercept_)
            results_reg["Coef"].append(lr.best_estimator_.coef_)
            results_reg['MAE'].append(lr.best_score_)
            results_reg['MAPE'].append(mean_absolute_percentage_error(dfteste["Vendas Reais Qtde"], 
                                                                      dfteste["VendasPrevistas"]))
            

    df_coeficientes = pd.DataFrame(results_reg)
    df_coeficientes = df_coeficientes.explode('Coef')
    df_coeficientes['Intercept'] = df_coeficientes['Intercept'].astype(float)
    num_coef = len(df_coeficientes["Coef"][0])
    a = np.array(df_coeficientes["Coef"].explode()).reshape(df_coeficientes.shape[0], num_coef)
    df__ = pd.DataFrame(a, columns = ["Coef1", "Coef2"])
    df_coeficientes = df_coeficientes.merge(df__, left_index = True, right_index = True)
    df_coeficientes = df_coeficientes.drop(columns = "Coef")
    
    df_coeficientes = df_coeficientes[["Serie", "Intercept", "Coef1", "Coef2", "MAE", "MAPE"]]
    
    #df = df.style.set_properties(**{'text-align': 'left'})
    #df_coeficientes = df_coeficientes.style.set_properties(**{'text-align': 'left'})
    
    results["Ajuste"].append(df)
    results["Previsoes"].append(df_coeficientes)
    
    df['VendasPrevistas'] = df['VendasPrevistas'].apply(lambda x: round(x))
    
    return df

In [6]:
df_predicoes = modelos(df_model, ['PrMedPonderado', 'CMV_QTD'])

In [8]:
df_predicoes.loc[(df_predicoes['NomeModelo'] == 'LR') & 
                 (df_predicoes['Vendas Reais Qtde'] < df_predicoes['VendasPrevistas']), 
                 ['UF_GrpProd', 'Vendas Reais Qtde', 'VendasPrevistas', 'NomeModelo']]\
                 .sort_values(by='Vendas Reais Qtde').head(60)

,UF_GrpProd,Vendas Reais Qtde,VendasPrevistas,NomeModelo
3762,DF-RP_CAMA,2.0,3,LR
4614,MT-RP_CAMA,2.0,3,LR
3694,PR-RP_CAMA,2.0,5,LR
4768,SC-RP_CAMA,2.0,3,LR
4534,SC-RP_CAMA,2.0,4,LR
4244,SC-RP_CAMA,2.0,5,LR
4112,SC-RP_CAMA,2.0,6,LR
3890,SC-RP_CAMA,2.0,5,LR
4562,PR-RP_CAMA,2.0,4,LR
4820,PR-RP_CAMA,2.0,8,LR


In [9]:
df_predicoes.groupby(['NomeModelo'])['NomeModelo'].count()

NomeModelo
LR     2859
XGB    1349
Name: NomeModelo, dtype: int64